In [127]:
import os
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from dateutil import parser
import io
import base64
from IPython.display import HTML
from imblearn.under_sampling import RandomUnderSampler

In [128]:
chunk_num = pd.read_csv("./train_numeric.csv",     chunksize = 100)
chunk_dat = pd.read_csv("./train_date.csv"   ,     chunksize = 100)
chunk_cat = pd.read_csv("./train_categorical.csv", chunksize = 100)

sample_num = 100000
df_num = chunk_num.get_chunk(sample_num)
df_dat = chunk_dat.get_chunk(sample_num)
df_cat = chunk_cat.get_chunk(sample_num)

C:\Users\Tomonobu\Anaconda3\lib\site-packages\ipykernel\zmqshell.py:533: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,41,42,43,44,45,46,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,188,189,190,191,192,193,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,324,335,337,340,342,344,346,348,351,354,357,360,363,366,369,372,375,378,381,384,387,390,393,396,399,402,

In [133]:
df_num.head()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,0
1,6,999.000,999.000,999.000,999.000,999.000,999.000,999.000,999.000,999.000,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,0


In [130]:
# 全部 NaN な列をdropして、Nanを大きな値で置換。
print("before" + str(df_num.shape))
df_num.dropna(how='all', axis=1, inplace=True)
df_num.fillna(999, inplace=True)
print("after" + str(df_num.shape))

before(100000, 970)
after(100000, 970)


In [134]:
# 全部 NaN な列をdropして、Nanを大きな値で置換。
print("before" + str(df_cat.shape))
df_cat.dropna(how='all', axis=1, inplace=True)
df_cat.fillna(999, inplace=True)
print("after" + str(df_cat.shape))

before(100000, 2141)
after(100000, 1128)


In [161]:
# カテゴリ変数のパース(仮実装で、値に変換するだけ)
# 最終的にはビットマップっぽいので、エンコーディングする。
from parse import *
def cat_data_to_val(x):
    if type(x) is str:
        r = parse("T{}", x)
        return float(r[0])
    else:
        return x


In [173]:
for column_name, item in df_cat.iteritems():
    item = item.map(lambda x: cat_data_to_val(x))


In [172]:
df_cat['L0_S9_F154'] = df_cat['L0_S9_F154'].map(lambda x: cat_data_to_val(x))
df_cat['L0_S9_F154'] .unique()

array([  9.99000000e+02,   8.00000000e+00,   4.00000000e+00,
         4.85760000e+04,   5.12000000e+02,   1.67772480e+07,
         9.60000000e+01,   4.80000000e+01,   1.67772320e+07,
         1.67775570e+07,   1.28000000e+02])

In [165]:
for column_name, item in df_cat.iteritems():
    print(column_name, item.unique())


Id [     4      6      7 ..., 200349 200351 200352]
L0_S1_F25 [999 'T1']
L0_S1_F27 [999 'T9']
L0_S1_F29 [999 'T1']
L0_S1_F31 [999 'T9']
L0_S2_F33 [999 'T1']
L0_S2_F35 [999 'T32' 'T48' 'T96' 'T64']
L0_S2_F37 [999 'T1']
L0_S2_F39 [999 'T32' 'T48' 'T96' 'T64']
L0_S2_F41 [999 'T1']
L0_S2_F43 [999 'T32' 'T48' 'T96' 'T64']
L0_S2_F45 [999 'T1']
L0_S2_F47 [999 'T32' 'T48' 'T96' 'T64']
L0_S2_F49 [999 'T1']
L0_S2_F51 [999 'T32' 'T48' 'T96' 'T64']
L0_S2_F53 [999 'T1']
L0_S2_F55 [999 'T32' 'T48' 'T96' 'T64']
L0_S2_F57 [999 'T1']
L0_S2_F59 [999 'T32' 'T48' 'T96' 'T64']
L0_S2_F61 [999 'T1']
L0_S2_F63 [999 'T32' 'T48' 'T96' 'T64']
L0_S2_F65 [999 'T1']
L0_S2_F67 [999 'T32' 'T48' 'T96' 'T64']
L0_S4_F105 [999 'T1']
L0_S4_F107 [999 'T1']
L0_S4_F108 [999 'T97' 'T98']
L0_S4_F110 [999 'T1']
L0_S4_F112 [999 'T1']
L0_S4_F113 [999 'T98']
L0_S9_F151 [999 'T1']
L0_S9_F153 [999 'T1']
L0_S9_F154 [999 'T8' 'T4' 'T48576' 'T512' 'T16777248' 'T96' 'T48' 'T16777232'
 'T16777557' 'T128']
L0_S9_F156 [999 'T1']
L0_S9_F158

L1_S24_F1766 [999 'T1']
L1_S24_F1767 [999 'T16777216' 'T331648']
L1_S24_F1769 [999 'T1']
L1_S24_F1771 [999 'T1']
L1_S24_F1772 [999 'T16777216']
L1_S24_F1774 [999 'T1']
L1_S24_F1776 [999 'T1']
L1_S24_F1777 [999 'T16777216' 'T331648']
L1_S24_F1779 [999 'T1']
L1_S24_F1781 [999 'T1']
L1_S24_F1782 [999 'T16777216' 'T331648']
L1_S24_F1784 [999 'T1']
L1_S24_F1786 [999 'T1']
L1_S24_F1787 [999 'T16777216' 'T331648']
L1_S24_F1789 [999 'T1']
L1_S24_F1791 [999 'T1']
L1_S24_F1792 [999 'T16777216' 'T331648']
L1_S24_F1794 [999 'T1']
L1_S24_F1796 [999 'T1']
L1_S24_F1797 [999 'T16777216' 'T331648']
L1_S24_F1799 [999 'T1']
L1_S24_F1801 [999 'T1']
L1_S24_F1802 [999 'T16777216' 'T331648']
L1_S24_F1804 [999 'T1']
L1_S24_F1806 [999 'T1']
L1_S24_F1807 [999 'T16777216' 'T331648']
L1_S24_F1827 [999 'T3']
L1_S25_F1852 [999 'T4' 'T5' 'T3' 'T6' 'T8' 'T2' 'T7']
L1_S25_F1882 [999 'T5']
L1_S25_F1901 [999 'T1']
L1_S25_F1903 [999 'T1']
L1_S25_F1904 [999 'T786432' 'T524288' 'T8912896' 'T262144']
L1_S25_F1905 [999 'T1']

In [159]:
df_cat.head()

,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
1,6,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
2,7,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
3,9,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999
4,11,999,999,999,999,999,999,999,999,999,...,999,999,999,999,999,999,999,999,999,999


In [100]:
print(df_num[df_num.isnull().any(axis=1)])

Empty DataFrame
Columns: [Id, L0_S0_F0, L0_S0_F2, L0_S0_F4, L0_S0_F6, L0_S0_F8, L0_S0_F10, L0_S0_F12, L0_S0_F14, L0_S0_F16, L0_S0_F18, L0_S0_F20, L0_S0_F22, L0_S1_F24, L0_S1_F28, L0_S2_F32, L0_S2_F36, L0_S2_F40, L0_S2_F44, L0_S2_F48, L0_S2_F52, L0_S2_F56, L0_S2_F60, L0_S2_F64, L0_S3_F68, L0_S3_F72, L0_S3_F76, L0_S3_F80, L0_S3_F84, L0_S3_F88, L0_S3_F92, L0_S3_F96, L0_S3_F100, L0_S4_F104, L0_S4_F109, L0_S5_F114, L0_S5_F116, L0_S6_F118, L0_S6_F122, L0_S6_F132, L0_S7_F136, L0_S7_F138, L0_S7_F142, L0_S8_F144, L0_S8_F146, L0_S8_F149, L0_S9_F155, L0_S9_F160, L0_S9_F165, L0_S9_F170, L0_S9_F175, L0_S9_F180, L0_S9_F185, L0_S9_F190, L0_S9_F195, L0_S9_F200, L0_S9_F205, L0_S9_F210, L0_S10_F219, L0_S10_F224, L0_S10_F229, L0_S10_F234, L0_S10_F239, L0_S10_F244, L0_S10_F249, L0_S10_F254, L0_S10_F259, L0_S10_F264, L0_S10_F269, L0_S10_F274, L0_S11_F282, L0_S11_F286, L0_S11_F290, L0_S11_F294, L0_S11_F298, L0_S11_F302, L0_S11_F306, L0_S11_F310, L0_S11_F314, L0_S11_F318, L0_S11_F322, L0_S11_F326, L0_S12_F33

In [101]:
import xgboost as xgb


def separate_X_y(df):
    X = df.drop(['Response'], axis=1)
    y = df['Response']
    return X, y

 

In [113]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

def train_with_r_forest(df):
    X_train, y_train = separate_X_y(df)

    rf = RandomForestClassifier(max_depth=5,n_estimators=10,oob_score=True)
    rf.fit(X_train, y_train)
    return rf





In [117]:
df_train, df_test = train_test_split(df_num)
rf = train_with_r_forest(df_train)

C:\Users\Tomonobu\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\Tomonobu\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [120]:
X_test, y_test = separate_X_y(df_test)
y_pred = rf.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [121]:

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[24833,     2],
       [  164,     1]], dtype=int64)

In [ ]:
df_desc.loc['max', np.isnan(df_desc.loc['max', :])]

In [123]:
df_cat

,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
df_num

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,999.0,999.0,999.0,999.0,999.00,999.0,999.0,999.0,999.0,0
1,6,999.000,999.000,999.000,999.000,999.000,999.000,999.000,999.000,999.000,...,999.0,999.0,999.0,999.0,999.00,999.0,999.0,999.0,999.0,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,999.0,999.0,999.0,999.0,999.00,999.0,999.0,999.0,999.0,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,999.0,999.0,999.0,999.0,999.00,999.0,999.0,999.0,999.0,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,999.0,999.0,999.0,999.0,999.00,999.0,999.0,999.0,999.0,0
5,13,0.003,0.019,0.294,0.312,0.031,0.161,0.022,0.088,-0.005,...,999.0,999.0,999.0,999.0,999.00,999.0,999.0,999.0,999.0,0
6,14,999.000,999.000,999.000,999.000,999.000,999.000,999.000,999.000,999.000,...,999.0,999.0,999.0,999.0,999.00,999.0,999.0,999.0,999.0,0
7,16,999.000,999.000,999.000,999.000,999.000,999.000,999.000,999.000,999.000,...,999.0,999.0,999.0,999.0,999.00,999.0,999.0,999.0,999.0,0
8,18,-0.016,-0.041,-0.179,-0.179,-0.056,0.161,-0.007,-0.032,-0.082,...,999.0,999.0,999.0,999.0,999.00,999.0,999.0,999.0,999.0,0
9,23,999.000,999.000,999.000,999.000,999.000,999.000,999.000,999.000,999.000,...,999.0,999.0,999.0,999.0,999.00,999.0,999.0,999.0,999.0,0
